In [2]:
#Calculate perticipation percentage
import pandas as pd
import h5py
import numpy as np
from scipy.signal import savgol_filter
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

wt_behavior_input = '/Users/donglinhan/Desktop/SLEAP/WT_LDfull_all_behavior_modified_again.xlsx'
het_behavior_input = '/Users/donglinhan/Desktop/SLEAP/Het_LD_all_behavior_modified_again.xlsx'
hom_behavior_input = '/Users/donglinhan/Desktop/SLEAP/PDF_LDfull_all_behavior_modified_again.xlsx'
wt_behaviors_original = pd.read_excel(wt_behavior_input)
het_behaviors_original = pd.read_excel(het_behavior_input)
hom_behaviors_original = pd.read_excel(hom_behavior_input)
wt_behaviors = wt_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)
het_behaviors = het_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)
hom_behaviors = hom_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)

WT_ZT_timepoints = {
    '0': range(0, 70),   
    '3': range(70, 150),  
    '6': range(150, 220), 
    '9': range(220, 300),
    '12': range(300, 380),
    '15': range(380, 460),
    '18': range(460, 530),
    '21': range(530, 600),
}

HET_ZT_timepoints = {
    '0': range(0, 60),   
    '3': range(60, 120),  
    '6': range(120, 180), 
    '9': range(180, 240),
    '12': range(240, 300),
    '15': range(300, 360),
    '18': range(360, 420),
    '21': range(420, 480),
}

HOM_ZT_timepoints = {
    '0': range(0, 58),   
    '3': range(58, 116),  
    '6': range(116, 174), 
    '9': range(174, 232),
    '12': range(232, 290),
    '15': range(290, 347),
    '18': range(347, 404),
    '21': range(404, 462),
}

wt_participation = []
het_participation = []
hom_participation = []

for ZT, cols in WT_ZT_timepoints.items():
    wt_behavior_data = wt_behaviors.iloc[:, cols]
    percentage_data = wt_behavior_data
    qualified = percentage_data.apply(
        lambda x: x[0:23].sum() == 0,
        axis=0
    ).sum()
    acute_responder_cols = percentage_data.apply(
        lambda x: (x[0:23].sum() == 0 and x[23:47].sum() > 0), 
        axis=0
    )
    acute_responder = acute_responder_cols.sum()
    participation = (acute_responder / qualified * 100)
    wt_participation.append(participation)

for ZT, cols in HET_ZT_timepoints.items():
    het_behavior_data = het_behaviors.iloc[:, cols]
    percentage_data = het_behavior_data
    qualified = percentage_data.apply(
        lambda x: x[0:23].sum() == 0,
        axis=0
    ).sum()
    acute_responder_cols = percentage_data.apply(
        lambda x: (x[0:23].sum() == 0 and x[23:47].sum() > 0), 
        axis=0
    )
    acute_responder = acute_responder_cols.sum()
    participation = (acute_responder / qualified * 100)
    het_participation.append(participation)

for ZT, cols in HOM_ZT_timepoints.items():
    hom_behavior_data = hom_behaviors.iloc[:, cols]
    percentage_data = hom_behavior_data
    qualified = percentage_data.apply(
        lambda x: x[0:23].sum() == 0,
        axis=0
    ).sum()
    
    acute_responder_cols = percentage_data.apply(
        lambda x: (x[0:23].sum() == 0 and x[23:47].sum() > 0), 
        axis=0
    )
    acute_responder = acute_responder_cols.sum()
    participation = (acute_responder / qualified * 100)
    hom_participation.append(participation)


print (wt_participation)
print (het_participation)
print (hom_participation)

[67.6923076923077, 60.86956521739131, 49.275362318840585, 61.33333333333333, 65.71428571428571, 0.0, 2.857142857142857, 20.0]
[81.35593220338984, 71.15384615384616, 66.07142857142857, 68.62745098039215, 70.83333333333334, 5.0, 15.0, 15.0]
[76.36363636363637, 76.59574468085107, 82.35294117647058, 85.41666666666666, 82.85714285714286, 8.771929824561402, 15.789473684210526, 27.586206896551722]


In [7]:
#LD acute vs sustained activity analysis
import pandas as pd
import h5py
import numpy as np
from scipy.signal import savgol_filter
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

wt_behavior_input = '/Users/donglinhan/Desktop/SLEAP/WT_LDfull_all_behavior_modified_again.xlsx'
het_behavior_input = '/Users/donglinhan/Desktop/SLEAP/Het_LD_all_behavior_modified_again.xlsx'
hom_behavior_input = '/Users/donglinhan/Desktop/SLEAP/PDF_LDfull_all_behavior_modified_again.xlsx'
wt_behaviors_original = pd.read_excel(wt_behavior_input)
het_behaviors_original = pd.read_excel(het_behavior_input)
hom_behaviors_original = pd.read_excel(hom_behavior_input)
wt_behaviors = wt_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)
het_behaviors = het_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)
hom_behaviors = hom_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)

wt_activity_input = '/Users/donglinhan/Desktop/SLEAP/WT_LDfull_all_collapsed_distances_300.xlsx'
het_activity_input = '/Users/donglinhan/Desktop/SLEAP/HET_LD_all_collapsed_distances_300.xlsx'
hom_activity_input = '/Users/donglinhan/Desktop/SLEAP/PDF_LDfull_all_collapsed_distances_300.xlsx'
wt_activities = pd.read_excel(wt_activity_input)
het_activities = pd.read_excel(het_activity_input)
hom_activities = pd.read_excel(hom_activity_input)

def bout_count_function(activity_series):
    bout_lengths = []
    current_bout = 0
    
    for activity in activity_series:
        if activity == 1:
            current_bout += 1
        else:
            if current_bout > 0:
                bout_lengths.append(current_bout)
            current_bout = 0

    if current_bout > 0:
        bout_lengths.append(current_bout)
    
    if bout_lengths:
        bout_count = len(bout_lengths)
    else:
        bout_count = 0
        
    return bout_count

def average_activity_bout_length(activity_series):
    bout_lengths = []
    current_bout = 0
    
    for activity in activity_series:
        if activity == 1:
            current_bout += 1
        else:
            if current_bout > 0:
                bout_lengths.append(current_bout)
            current_bout = 0
    
    if current_bout > 0:
        bout_lengths.append(current_bout)
    
    if bout_lengths:
        average_length = sum(bout_lengths)/len(bout_lengths)
    else:
        average_length = 0
        
    return average_length

def median_active_time(activity, time_points):
    # Identify active times where activity is 1 (walking) or 2 (flying)
    active_times = time_points[(activity == 1) | (activity == 2)]
    if len(active_times) > 0:
        return np.median(active_times) / 60  # Convert seconds to minutes
    else:
        return np.nan  # No active times


WT_ZT_timepoints = {
    '0': range(0, 70),   
    '3': range(70, 150),  
    '6': range(150, 220), 
    '9': range(220, 300),
    '12': range(300, 380),
    '15': range(380, 460),
    '18': range(460, 530),
    '21': range(530, 600),
}

HET_ZT_timepoints = {
    '0': range(0, 60),   
    '3': range(60, 120),  
    '6': range(120, 180), 
    '9': range(180, 240),
    '12': range(240, 300),
    '15': range(300, 360),
    '18': range(360, 420),
    '21': range(420, 480),
}

HOM_ZT_timepoints = {
    '0': range(0, 58),   
    '3': range(58, 116),  
    '6': range(116, 174), 
    '9': range(174, 232),
    '12': range(232, 290),
    '15': range(290, 347),
    '18': range(347, 404),
    '21': range(404, 462),
}

wt_total_acute_activity = pd.DataFrame(index = range(100)) 
wt_total_sustained_activity_4 = pd.DataFrame(index = range(100)) 
wt_total_bout_count = pd.DataFrame(index = range(100)) 
wt_total_average_bout = pd.DataFrame(index = range(100)) 
wt_total_median_time = pd.DataFrame(index = range(100)) 

het_total_acute_activity = pd.DataFrame(index = range(100)) 
het_total_sustained_activity_4 = pd.DataFrame(index = range(100)) 
het_total_bout_count = pd.DataFrame(index = range(100)) 
het_total_average_bout = pd.DataFrame(index = range(100)) 
het_total_median_time = pd.DataFrame(index = range(100)) 

hom_total_acute_activity = pd.DataFrame(index = range(100)) 
hom_total_sustained_activity_4 = pd.DataFrame(index = range(100)) 
hom_total_bout_count = pd.DataFrame(index = range(100)) 
hom_total_average_bout = pd.DataFrame(index = range(100)) 
hom_total_median_time = pd.DataFrame(index = range(100)) 

time_points = np.arange(0, 600, 5)

for ZT, cols in WT_ZT_timepoints.items():
    wt_behavior_data = wt_behaviors.iloc[:, cols]
    wt_activity_data = wt_activities.iloc[:,cols]
    wt_behavior_type = wt_behaviors_original.iloc[:, cols]
    wt_acute_activity = []
    wt_sustained_activity_4 = []
    wt_bout_count = []
    wt_average_bout = []
    wt_median_time = []
    for i in wt_behavior_data:
        if wt_behavior_data[i][23:47].sum() > 0 and wt_behavior_data[i][0:23].sum() == 0: 
            sustained_activity_4 = wt_activity_data[i][95:143].sum()  / 240
            bout_count = bout_count_function(pd.Series(wt_behavior_data[i]))
            average_bout = average_activity_bout_length(pd.Series(wt_behavior_data[i]))
            acute_activity = wt_activity_data[i][23:47].sum() / 120
            median_time = median_active_time(wt_behavior_type[i][23:143], time_points)
        elif wt_behavior_data[i][23:47].sum() == 0 and wt_behavior_data[i][0:23].sum() == 0: 
            sustained_activity_4 = None
            bout_count = None
            average_bout = None
            acute_activity = wt_activity_data[i][23:47].sum() / 120
            median_time = None
        else:
            sustained_activity_4 = None
            bout_count = None
            average_bout = None
            acute_activity = None
            median_time = None
        wt_sustained_activity_4.append(sustained_activity_4)
        wt_bout_count.append(bout_count)
        wt_average_bout.append(average_bout)
        wt_acute_activity.append(acute_activity)
        wt_median_time.append(median_time)
    wt_total_sustained_activity_4[ZT] = pd.Series(wt_sustained_activity_4)
    wt_total_bout_count[ZT] = pd.Series(wt_bout_count)
    wt_total_average_bout[ZT] = pd.Series(wt_average_bout)
    wt_total_acute_activity[ZT] = pd.Series(wt_acute_activity)
    wt_total_median_time[ZT] = pd.Series(wt_median_time)
    

for ZT, cols in HET_ZT_timepoints.items():
    het_behavior_data = het_behaviors.iloc[:, cols]
    het_activity_data = het_activities.iloc[:,cols]
    het_behavior_type = het_behaviors_original.iloc[:, cols]
    het_sustained_activity_4 = []
    het_bout_count = []
    het_acute_activity = []
    het_average_bout = []
    het_median_time = []
    for i in het_behavior_data:
        if het_behavior_data[i][23:47].sum() > 0 and het_behavior_data[i][0:23].sum() == 0: 
            sustained_activity_4 = het_activity_data[i][95:143].sum()  / 240
            bout_count = bout_count_function(pd.Series(het_behavior_data[i]))
            average_bout = average_activity_bout_length(pd.Series(het_behavior_data[i]))
            acute_activity = het_activity_data[i][23:47].sum() / 120
            median_time = median_active_time(het_behavior_type[i][23:143], time_points)
        elif het_behavior_data[i][23:47].sum() == 0 and het_behavior_data[i][0:23].sum() == 0: 
            sustained_activity_4 = None
            bout_count = None
            average_bout = None
            acute_activity = het_activity_data[i][23:47].sum() / 120
            median_time = None
        else:
            sustained_activity_4 = None
            bout_count = None
            average_bout = None
            acute_activity = None
            median_time = None
        het_sustained_activity_4.append(sustained_activity_4)
        het_bout_count.append(bout_count)
        het_average_bout.append(average_bout)
        het_acute_activity.append(acute_activity)
        het_median_time.append(median_time)
    het_total_sustained_activity_4[ZT] = pd.Series(het_sustained_activity_4)
    het_total_bout_count[ZT] = pd.Series(het_bout_count)
    het_total_average_bout[ZT] = pd.Series(het_average_bout)
    het_total_acute_activity[ZT] = pd.Series(het_acute_activity)
    het_total_median_time[ZT] = pd.Series(het_median_time)

for ZT, cols in HOM_ZT_timepoints.items():
    hom_behavior_data = hom_behaviors.iloc[:, cols]
    hom_activity_data = hom_activities.iloc[:,cols]
    hom_behavior_type = hom_behaviors_original.iloc[:, cols]
    hom_sustained_activity_4 = []
    hom_bout_count = []
    hom_acute_activity = []
    hom_average_bout = []
    hom_median_time = []
    for i in hom_behavior_data:
        if hom_behavior_data[i][23:47].sum() > 0 and hom_behavior_data[i][0:23].sum() == 0: 
            sustained_activity_4 = hom_activity_data[i][95:143].sum() / 240
            bout_count = bout_count_function(pd.Series(hom_behavior_data[i]))
            average_bout = average_activity_bout_length(pd.Series(hom_behavior_data[i]))
            acute_activity = hom_activity_data[i][23:47].sum() / 120
            median_time = median_active_time(hom_behavior_type[i][23:143], time_points)
        elif hom_behavior_data[i][23:47].sum() == 0 and hom_behavior_data[i][0:23].sum() == 0: 
            sustained_activity_4 = None
            bout_count = None
            average_bout = None
            acute_activity = hom_activity_data[i][23:47].sum() / 120
            median_time = None
        else:  
            sustained_activity_4 = None
            bout_count = None
            average_bout = None
            acute_activity = None
            median_time = None
        hom_sustained_activity_4.append(sustained_activity_4)
        hom_bout_count.append(bout_count)
        hom_average_bout.append(average_bout)
        hom_acute_activity.append(acute_activity)
        hom_median_time.append(median_time)
    hom_total_sustained_activity_4[ZT] = pd.Series(hom_sustained_activity_4)
    hom_total_bout_count[ZT] = pd.Series(hom_bout_count)
    hom_total_average_bout[ZT] = pd.Series(hom_average_bout)
    hom_total_acute_activity[ZT] = pd.Series(hom_acute_activity)
    hom_total_median_time[ZT] = pd.Series(hom_median_time)


wt_total_sustained_activity_4_output_file_path = 'persistence_analysis/wt_ld_total_sustained_activity_last_4_min.xlsx'
wt_total_bout_count_output_file_path = 'persistence_analysis/wt_ld_total_bout_count.xlsx'
wt_total_average_bout_output_file_path = 'persistence_analysis/wt_ld_total_average_bout.xlsx'
wt_total_acute_activity_output_file_path = 'persistence_analysis/wt_ld_total_acute_activity.xlsx'
wt_total_median_time_output_file_path = 'persistence_analysis/wt_ld_total_median_time.xlsx'


het_total_sustained_activity_4_output_file_path = 'persistence_analysis/het_ld_total_sustained_activity_last_4_min.xlsx'
het_total_bout_count_output_file_path = 'persistence_analysis/het_ld_total_bout_count.xlsx'
het_total_average_bout_output_file_path = 'persistence_analysis/het_ld_total_average_bout.xlsx'
het_total_acute_activity_output_file_path = 'persistence_analysis/het_ld_total_acute_activity.xlsx'
het_total_median_time_output_file_path = 'persistence_analysis/het_ld_total_median_time.xlsx'


hom_total_sustained_activity_4_output_file_path = 'persistence_analysis/hom_ld_total_sustained_activity_last_4_min.xlsx'
hom_total_bout_count_output_file_path = 'persistence_analysis/hom_ld_total_bout_count.xlsx'
hom_total_average_bout_output_file_path = 'persistence_analysis/hom_ld_total_average_bout.xlsx'
hom_total_acute_activity_output_file_path = 'persistence_analysis/hom_ld_total_acute_activity.xlsx'
hom_total_median_time_output_file_path = 'persistence_analysis/hom_ld_total_median_time.xlsx'

wt_total_sustained_activity_4.to_excel(wt_total_sustained_activity_4_output_file_path, index=False)
wt_total_bout_count.to_excel(wt_total_bout_count_output_file_path, index=False)
wt_total_average_bout.to_excel(wt_total_average_bout_output_file_path, index=False)
wt_total_acute_activity.to_excel(wt_total_acute_activity_output_file_path, index=False)
wt_total_median_time.to_excel(wt_total_median_time_output_file_path, index=False)


het_total_sustained_activity_4.to_excel(het_total_sustained_activity_4_output_file_path, index=False)
het_total_bout_count.to_excel(het_total_bout_count_output_file_path, index=False)
het_total_average_bout.to_excel(het_total_average_bout_output_file_path, index=False)
het_total_acute_activity.to_excel(het_total_acute_activity_output_file_path, index=False)
het_total_median_time.to_excel(het_total_median_time_output_file_path, index=False)


hom_total_sustained_activity_4.to_excel(hom_total_sustained_activity_4_output_file_path, index=False)
hom_total_bout_count.to_excel(hom_total_bout_count_output_file_path, index=False)
hom_total_average_bout.to_excel(hom_total_average_bout_output_file_path, index=False)
hom_total_acute_activity.to_excel(hom_total_acute_activity_output_file_path, index=False)
hom_total_median_time.to_excel(hom_total_median_time_output_file_path, index=False)

In [8]:
#DD activity
import pandas as pd
import h5py
import numpy as np
from scipy.signal import savgol_filter
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

wt_behavior_input = '/Users/donglinhan/Desktop/SLEAP/WT_DD2_all_behavior_modified_again.xlsx'
hom_behavior_input = '/Users/donglinhan/Desktop/SLEAP/PDF_DD2_all_behavior_modified_again.xlsx'
wt_behaviors_original = pd.read_excel(wt_behavior_input)
hom_behaviors_original = pd.read_excel(hom_behavior_input)
wt_behaviors = wt_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)
hom_behaviors = hom_behaviors_original.applymap(lambda x: 1 if x != 0 else 0)
wt_activity_input = '/Users/donglinhan/Desktop/SLEAP/WT_DD2_collapsed_distances_300.xlsx'
hom_activity_input = '/Users/donglinhan/Desktop/SLEAP/PDF_DD2_all_collapsed_distances_300.xlsx'
wt_activities = pd.read_excel(wt_activity_input)
hom_activities = pd.read_excel(hom_activity_input)

WT_CT_timepoints = {
    '0': range(0, 30),   
    '3': range(30, 60),  
    '6': range(60, 90), 
    '9': range(90, 120),
    '12': range(120, 150),
    '15': range(150, 180),
    '18': range(180, 210),
    '21': range(210, 240),
}

HOM_CT_timepoints = {
    '0': range(0, 29),   
    '3': range(29, 58),  
    '6': range(58, 87), 
    '9': range(87, 116),
    '12': range(116, 145),
    '15': range(145, 174),
    '18': range(174, 203),
    '21': range(203, 232),
}


wt_total_acute_activity = pd.DataFrame(index = range(100)) 
hom_total_acute_activity = pd.DataFrame(index = range(100)) 

for CT, cols in WT_CT_timepoints.items():
    wt_behavior_data = wt_behaviors.iloc[:, cols]
    wt_activity_data = wt_activities.iloc[:,cols]
    wt_behavior_type = wt_behaviors_original.iloc[:, cols]
    wt_acute_activity = []
    for i in wt_behavior_data:
        if wt_behavior_data[i][23:47].sum() > 0 and wt_behavior_data[i][0:23].sum() == 0: 
            acute_activity = wt_activity_data[i][23:47].sum() / 120
        elif wt_behavior_data[i][23:47].sum() == 0 and wt_behavior_data[i][0:23].sum() == 0: 
            acute_activity = wt_activity_data[i][23:47].sum() / 120
        else:
            acute_activity = None
        wt_acute_activity.append(acute_activity)
    wt_total_acute_activity[CT] = pd.Series(wt_acute_activity)

for CT, cols in HOM_CT_timepoints.items():
    hom_behavior_data = hom_behaviors.iloc[:, cols]
    hom_activity_data = hom_activities.iloc[:,cols]
    hom_behavior_type = hom_behaviors_original.iloc[:, cols]
    hom_acute_activity = []
    for i in hom_behavior_data:
        if hom_behavior_data[i][23:47].sum() > 0 and hom_behavior_data[i][0:23].sum() == 0: 
            acute_activity = hom_activity_data[i][23:47].sum() / 120
        elif hom_behavior_data[i][23:47].sum() == 0 and hom_behavior_data[i][0:23].sum() == 0: 
            acute_activity = hom_activity_data[i][23:47].sum() / 120
        else:
            acute_activity = None
        hom_acute_activity.append(acute_activity)
    hom_total_acute_activity[CT] = pd.Series(hom_acute_activity)


wt_total_acute_activity_output_file_path = 'persistence_analysis/wt_dd_total_acute_activity.xlsx'
hom_total_acute_activity_output_file_path = 'persistence_analysis/hom_dd_total_acute_activity.xlsx'

wt_total_acute_activity.to_excel(wt_total_acute_activity_output_file_path, index=False)
hom_total_acute_activity.to_excel(hom_total_acute_activity_output_file_path, index=False)